# Libraries

In [1]:
import sys
import numpy as np
import pandas as pd
import os
os.environ["OBJC_DISABLE_INITIALIZE_FORK_SAFETY"] = "YES"
sys.path.append("../")

In [2]:
df = pd.read_csv('../data/raw/properati_entrenamiento.csv')

In [3]:
df.head(2)

,id,ad_type,start_date,end_date,created_on,lat,lon,l1,l2,l3,...,bathrooms,surface_total,surface_covered,price,currency,price_period,title,description,property_type,operation_type
0,1,Propiedad,2019-07-07,2019-07-10,2019-07-07,NaN,NaN,Argentina,Buenos Aires Interior,Urquiza,...,NaN,NaN,NaN,3800.0,ARS,Mensual,COCHERA EN ALQUILER,Xintel (SLC-SI1-560) Alquiler de Cochera en CA...,Cochera,Alquiler
1,2,Propiedad,2019-07-07,9999-12-31,2019-07-07,-64.184812,-31.421307,Argentina,Córdoba,Córdoba,...,NaN,12.0,12.0,16000.0,USD,Mensual,Cochera venta,VENDO COCHERA- EXCELENTE UBICACIÓN! Calle Buen...,Cochera,Venta


In [4]:
df.isna().sum()

id                      0
ad_type                 0
start_date              0
end_date                0
created_on              0
lat                153174
lon                152240
l1                      0
l2                      0
l3                  57286
l4                 768768
l5                 989289
l6                 994000
rooms              489672
bedrooms           602620
bathrooms          219251
surface_total      542896
surface_covered    554230
price               44381
currency            47251
price_period       625976
title                  28
description            22
property_type           0
operation_type          0
dtype: int64

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 994000 entries, 0 to 993999
Data columns (total 25 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   id               994000 non-null  int64  
 1   ad_type          994000 non-null  object 
 2   start_date       994000 non-null  object 
 3   end_date         994000 non-null  object 
 4   created_on       994000 non-null  object 
 5   lat              840826 non-null  float64
 6   lon              841760 non-null  float64
 7   l1               994000 non-null  object 
 8   l2               994000 non-null  object 
 9   l3               936714 non-null  object 
 10  l4               225232 non-null  object 
 11  l5               4711 non-null    object 
 12  l6               0 non-null       float64
 13  rooms            504328 non-null  float64
 14  bedrooms         391380 non-null  float64
 15  bathrooms        774749 non-null  float64
 16  surface_total    451104 non-null  floa

In [6]:
from src.utils.etl.helper import filter_columns, fill_na, clean_text
from src.constants import MAP_PROVINCE

In [7]:
df = filter_columns(df)

In [8]:
df.isna().sum()

id                       0
ad_type                  0
lat                  11350
lon                  11238
country                  0
province                 0
suburb                5416
published_suburb    122726
rooms                10756
bedrooms             40719
bathrooms             6743
surface_total        45196
surface_covered      42970
price                    0
currency                 0
title                   11
description              1
property_type            0
operation_type           0
dtype: int64

In [9]:
df = fill_na(df)

In [10]:
df.isna().sum()

id                       0
ad_type                  0
lat                  11350
lon                  11350
country                  0
province                 0
suburb                5416
published_suburb    122726
rooms                10756
bedrooms             44388
bathrooms             6743
surface_total        45232
surface_covered      43097
price                    0
currency                 0
title                    0
description              0
property_type            0
operation_type           0
dtype: int64

In [11]:
df[["lat", "lon"]] = df[["lon", "lat"]]
df["province"] = df["province"].map(MAP_PROVINCE)

In [12]:
from multiprocessing import  Pool
import numpy as np

def parallelize_dataframe(df, func, n_cores=10):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [13]:
df = parallelize_dataframe(df, clean_text)

# df = clean_text(df, n_partitions=128)

KeyboardInterrupt: 